# Pyperclip

## Create LaTeX table

In [ ]:
import pyperclip

In [ ]:
# load stuff that seldom changes
label = 'text'
caption = 'text'
bht = '[!bht]'
lp = 'l'
midrule = ''
breplace_sz = True

In [ ]:
# adapt table here if you want ######################################
#"""
caption = 'Standardkonturen und ihre Verwendung'
label = 'standkont' #"""
"""
bht = '' #"""
"""
lp = 'p{cm}' #"""
"""
midrule = '\n\\midrule' #"""

# create table ######################################################
table_str = pyperclip.paste()
if breplace_sz:
    table_str = table_str.replace("ß","{\ss}")

# avoid last empty row
if table_str[-1] == "\n":
    table_str = table_str[:-1]

table_list = table_str.split("\n")
table_arr = [[cell.strip() for cell in row.split("\t")] for row in table_list]
table_start = '\\begin{table}%s\n\\begin{tabular}{%s}\n\\toprule\n%s \\\\\n\\midrule' % (
    bht,
    "|".join([lp]*len(table_arr[0])),
    " & ".join(['\\textbf{%s}' % col_name for col_name in table_arr[0]])
)
table_body = "\n".join(["%s \\\\%s" % (" & ".join(row),midrule) for row in table_arr[1:]])
table_end = "\\end{tabular}\n\\caption{%s}\n\\label{tab:%s}\n\\end{table}\n"%(caption, label)
table = "\n".join([table_start, table_body, table_end])
print(table)

In [ ]:
pyperclip.copy(table)

## Create HTML table

In [ ]:
import pyperclip

In [ ]:
# adapt table here if you want, beware of missing spaces ############
table_id = ''  # id=
table_classes = ''  # class=''
th_id = ''  # id=
th_classes = ''  # class=''
tr_id = ''  # id=
tr_classes = ''  # class=''
td_id = ''  # id=
td_classes = ''  # class=''

# create table ######################################################
table_str = pyperclip.paste()

# avoid last empty row
if table_str[-1] == "\n":
    table_str = table_str[:-1]
    
table_list = table_str.split("\n")
table_arr = [[cell.strip() for cell in row.split("\t")] for row in table_list]
table_columns = ["\t<th%s%s>%s</th>\n" % (th_id, th_classes, th) for th in table_arr[0]] \
    + ["\t<tr%s%s>\n%s\t</tr>\n" % (
        tr_id,
        tr_classes,
        "".join(["\t\t<td%s%s>%s</td>\n" % (td_id, td_classes, tr) for tr in row])
    )
    for row in table_arr[1:]
]
table = "<table%s%s>\n%s</table>\n" % (table_id, table_classes, "".join(table_columns))
print(table)

In [ ]:
pyperclip.copy(table)

## Sort values separated by an inner cell delimiter in a column
e.g.
<table>
    <th>Column</th><th><div align=center>&rarr;</div></th><th>Column</th>
    <tr><td><div align=center>bla</div></td><td><div align=center>&rarr;</div></td><td><div align=center>bla</div></td></tr>
    <tr><td>blo, ble</td><td><div align=center>&rarr;</div></td><td>ble, blo</td></tr>
</table>

In [ ]:
import pyperclip

In [ ]:
# adapt delimiter if needed
delimiter = "/"

column_str = pyperclip.paste()
column = column_str[:-1].split("\n")
column_new = []
for row in column:
    if row.find(delimiter) > -1:
        values = row.split(delimiter)
        column_new.append("/".join(sorted([values[0][1:]]+values[1:], key=lambda v: v.upper())))
    else:
        column_new.append(row)
column_str_new = "\n".join(column_new)
pyperclip.copy(column_str_new)
column_new

## Sort columns of a table individually

In [ ]:
import pyperclip

In [ ]:
table_str = pyperclip.paste()

# avoid last empty row
if table_str[-1] == "\n":
    table_str = table_str[:-1]

table_list = table_str.split("\n")
table_arr = [[cell.strip() for cell in row.split("\t")] for row in table_list]

# € has Unicode value 8364, after that the symbols are certainly irrelevant
sorted_columns = [sorted(list(column)[1:], key=lambda x: x.upper() if x != "" else "€")
                          for column in zip(*table_arr)]
table = [table_arr[0]] \
          + [[cell if cell != "" else "" for cell in column]
           for column in zip(*sorted_columns)
           if column != ('',)*len(table_arr[0])
          ]
table

In [ ]:
table_str = "\n".join(["\t".join(row) for row in table])
pyperclip.copy(table_str)

## Sort 2-dimensional row-array based on unique values in one column

e.g.

<table>
    <thead>
        <th>Before</th>
        <th>&rarr;</th>
        <th>After</th>
    </thead>
    <tbody>
        <tr>
            <td>[[1, "def", ...],</td>
            <td>&rarr;</td>
            <td>[[1, "def", ...],</td>
        </tr>
        <tr>
            <td>[1, "def", ...],</td>
            <td>&mdash;</td>
            <td>[2, "def", ...],</td>
        </tr>
        <tr>
            <td>[2, "def", ...],</td>
            <td>&#8599;</td>
            <td>[3, "def", ...],</td>
        </tr>
        <tr>
            <td>[3, "def", ...],</td>
            <td>&#8599;</td>
            <td>[4, "def", ...],</td>
        </tr>
        <tr>
            <td>[4, "def", ...],</td>
            <td>&#8599;</td>
            <td>...]</td>
        </tr>
        <tr>
            <td>...]</td>
            <td>&#8599;</td>
            <td></td>
        </tr>
    </tbody>
</table>

In [ ]:
import numpy as np
import pyperclip

def arr_dimen(a):
    # http://stackoverflow.com/questions/17531796/ddg#17532024
    return [len(a)] + arr_dimen(a[0]) if(type(a) == list) else []

def set_of_col_in_2darr(bdarray, index):
    """
    removes duplicates from the nth value of every subarray/-list/-tuple
    where duplicates are the 2nd occurrence onwards of a value
    needs: arr_dimen()
    
    bdarray: a bidimensional array or list or tuple
    index: index of the nth value (like a column) that should be filtered
    """
    
    # get unique values
    bdarray_type = type(bdarray)
    if (bdarray_type ==  list or bdarray_type == tuple) and len(arr_dimen(bdarray)) == 2:
        filter_column = np.array([a[index] for a in bdarray])
        bdarray = np.array(bdarray)
    elif bdarray_type == np.ndarray and len(bdarray.shape) == 2:
        filter_column = np.transpose(bdarray)[index]
    else:
        return    

    # filter original iterable
    filt = np.full(filter_column.shape[0], False)
    unique_positions = np.unique(filter_column, return_index=True)[1]
    filt[unique_positions] = True
    bd_array_filt = bdarray[filt]

    if bdarray_type == list:
        return bd_array_filt.tolist()
    elif bdarray_type == tuple:
        return tuple(map(tuple, bd_array_filt))
    else:
        return bd_array_filt

In [ ]:
bdarray = pyperclip.paste()
bdarray_filtered = set_of_col_in_2darr(bdarray, index)
print(len(bdarray), len(bdarray_filtered))

In [ ]:
pyperclip.copy(sorted(list(list_set)))

## Count unique values

In [ ]:
import pyperclip

In [ ]:
list_str = pyperclip.paste()

# avoid last empty row
if list_str[-1] == "\n":
    list_str = list_str[:-1]
    
list_set = set(list_str.split("\n"))
print(len(list_set))
pyperclip.copy(sorted(list(list_set)))

# MySQL dump to CSV file

In [ ]:
import pandas as pd
import os
import glob
import ast

path_mysql_file = '/mnt/e/Ganz_doll_wichtiges/DFKI/Masterarbeit/Datenbank/'
files = [file for file in glob.glob(path_mysql_file + 'MySQL/*') if file.find('/Dump') >= 0]
current_file = max(files, key=os.path.getmtime) # or getctime (last metadata change on linux)
table_name = ''

with open(current_file, 'r') as d:
    sql_dump = d.readlines()
#sql_dump[:40]

In [ ]:
values = []
all_values = []
table_names = []
column_names = []
all_column_names = []
bcol_names_start = False
bvalues_start = False
if table_name:
    table_names.append(table_name)

for line in sql_dump:

    # column description starts with this line
    if line.startswith('CREATE TABLE '):
        if not table_name:
            table_names.append(line.split('`')[1])
            bcol_names_start = True
        elif line.find(table_name) > 0:
            table_names.append(line.split('`')[1])
            bcol_names_start = True

    # parse column names or find the end of the column description
    elif bcol_names_start:
        if line.find('PRIMARY KEY') > 0 or line.startswith(')'):
            bcol_names_start = False
        else:
            column_names.append(line.split('`')[1])

    # parse values in this line
    elif line.startswith('INSERT INTO '):
        
        # python has problems to compare ` with its pendant in the file -> find()
        if not table_name or 12 < line.find(table_name) < 14:
            bvalues_start = True
            values_str = '[' + line[line.find('VALUES (') + 7:-2] + ']'
            values.extend([
                
                # pandas needs lists instead of tuples
                list(value) for value
                
                # 2 times .replace(',NULL,',',None,')\ for full recursivity
                in ast.literal_eval(values_str\
                                    .replace(',NULL,',',None,')\
                                    .replace(',NULL,',',None,')\
                                    .replace('(NULL,','(None,')\
                                    .replace(',NULL)',',None)')
                                   )
            ])
    
    # add the information at the end of the "INSERT INTO" block/line
    elif bvalues_start and line == "UNLOCK TABLES;\n":
        bvalues_start = False
        all_values.append(values)
        values = []
        all_column_names.append(column_names)
        column_names = []
        
    # if there is no "INSERT INTO" block/line, ignore the table
    elif not bvalues_start and line == "UNLOCK TABLES;\n" and table_names:
        del table_names[-1]
        column_names = []

assert len(table_names) == len(all_column_names) == len(all_values), \
    "table_names (%i), all_column_names (%i) and all_values (%i) don't have the same length"\
    % (len(table_names), len(all_column_names), len(all_values))

In [ ]:
# create/save the file(s)
for table_name, column_names, values in zip(table_names, all_column_names, all_values):
    dfsql_dump = pd.DataFrame(values, columns=column_names)

    # \r leads to strange behaviour when opening the csv with a program like LibreOffice Calc
    # pandas stores strings as a type called "object"
    # using replace on a df without any string values -> error
    if "object" in {str(d) for d in dfsql_dump.dtypes}:
        dfsql_dump.replace({'\r': '\n'}, regex=True, inplace=True)

    dfsql_dump.to_csv(path_mysql_file + "MySQL/" + table_name + '_mysql.tsv', sep='\t',
                      index=False)

# List TODOs from a LaTeX document

In [ ]:
import re

In [ ]:
texdocument =  "/mnt/e/Ganz_doll_wichtiges/DFKI/Masterarbeit/LaTeX/Archiv/Masterarbeit 03.01.19.tex"
with open(texdocument, "r") as d:
    latex_doc = d.readlines()

In [ ]:
todos = []
icurr_ch = 0
icurr_se = 0

def find_end(text, iend):
    searchtext = text[:iend]
    ibrackets_start = searchtext.count("{")
    ibrackets_end = searchtext.count("}")

    # if the searchtext is the whole todo note
    if ibrackets_start == ibrackets_end:
        return searchtext

    if ibrackets_start > ibrackets_end and text[iend + 1:].find("}") > -1:
        return find_end(text, iend + 2 + text[iend + 1:].find("}"))
    else:
        return "!?Error?!: %s" % searchtext

for line_nr, line in enumerate(latex_doc):
    if not line.isspace():

        # fill dictionary with chapters and sections
        if line.startswith("\\chapter"):
            icurr_ch += 1
            icurr_se = 0
        if line.startswith("\\section{"):
            icurr_se += 1

        istart = line.find("\\todo")
        if istart >= 0:

            # if the line is wholly a comment
            if line.startswith("%"):
                continue

            # erase the comment if there is one
            icomment_start = re.search(r"[^\\]%",line)
            if icomment_start:
                line = line[:icomment_start.start()]

            elements = []
            for el in line.split("\\todo")[1:]:
                elements.append(find_end(el, 1 + el.find("}")))
            if elements:
                todos.append([line_nr,float("%i.%i"%(icurr_ch,icurr_se)), " || ".join(elements)])

print(len(todos))
todos

In [ ]:
# count the words of the todo notes
len([word for line, chapter, text in todos
     for word in text.replace("} || {"," ").split()
    ])